In [ ]:
#default_exp modelling.mybifacialvf
#default_cls_lvl 3

In [ ]:
#export
from capetools.imports import *
from capetools.utils.tmy import read_tmy
import bifacialvf
import pvlib

In [ ]:
from nbdev.showdoc import show_doc

# bifacialvf
> A wrapper around NREL's bifacialvf, you need my version of [bifacialvf](http://github.com/tcapelle/bifacialvf). 

PLease install using 
```
pip install git+https://github.com/tcapelle/bifacialvf
``` 
before using this notebook.
It ads various very necessary functionalities to bifacialvf:
    - Uses pandas `DataFrames`'s as input data, through the use of `smimulate_inner` function
    - Uses timestamps on files, instead of the Year, Month, Day... splited dates
    - It is faster

Let's put some gps data:

In [ ]:
#export
CHAMBERY = {'Name':'Chambery', 'Latitude': 45.637001, 'Longitude': 5.881, 'Elevation': 235.0, 'TZ':-1.0}

bifacialvf has hard-coded column names 🤣

In [ ]:
DATAPATH = Path.cwd().parent/'data'
DATAPATH.ls()

(#2) [Path('/home/tc256760/Documents/capetools/data/pvgis_tmy_chambery.csv'),Path('/home/tc256760/Documents/capetools/data/sample_data.hdf')]

In [ ]:
fname = 'sample_data.hdf'
df = pd.read_hdf(DATAPATH/fname)
df.head()

,ghi,dni,dhi,temp_air,wind_speed,wind_dir,pressure,humidity,zenith,elevation,azimuth
Date UTC,,,,,,,,,,,
2019-01-01 00:00:00,-0.83,0.0,-0.83,4.39,1.87,230.1,1004.92,86.8,157.029,-67.029,12.054
2019-01-01 01:00:00,-0.93,0.0,-0.83,4.33,2.87,228.9,1004.94,86.8,152.143,-62.143,42.533
2019-01-01 02:00:00,-0.83,0.0,-0.83,4.29,2.86,229.4,1004.68,86.7,143.742,-53.742,63.390
2019-01-01 03:00:00,-0.83,0.0,-0.83,4.15,2.04,186.0,1003.99,85.7,133.848,-43.848,78.092
2019-01-01 04:00:00,-0.72,0.0,-0.83,3.85,1.91,200.9,1003.76,87.5,123.435,-33.435,89.729


In [ ]:
#export
def rename_cols(df: DataFrame, cols: list=['dni', 'dhi', 'zenith', 'azimuth', 'elevation']):
    "Rename to upper case dni and ghi cols"
    assert set(cols).issubset(df.columns.str.lower()), 'Missing columns on df'
    if not set(['DNI', 'DHI']).issubset(df.columns): 
        tmy3 = (df[cols]
                .rename(columns={'dni':'DNI', 'dhi':'DHI'}))
        return tmy3
    else: return df

In [ ]:
df.columns

Index(['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed', 'wind_dir', 'pressure',
       'humidity', 'zenith', 'elevation', 'azimuth'],
      dtype='object')

In [ ]:
data = rename_cols(df)

We will need to compare to pvfactors output, so we create a function to do that:

In [ ]:
#export
def format_output_as_pvfactor(res: DataFrame, cuts: int): 
    "Formats output from bifacialvf as pvfactor's one"
    front_cols = [f'No_{i+1}_RowFrontGTI' for i in range(cuts)]
    back_cols = [f'No_{i+1}_RowBackGTI' for i in range(cuts)]
    aux = pd.DataFrame(index=res.index)
    aux['qinc_front'] = res[front_cols].mean(axis=1)
    aux['qinc_back'] = res[back_cols].mean(axis=1)
    aux[back_cols] = res[back_cols]
    aux =  aux.rename(columns=dict(zip(back_cols,[f'qinc_back_{i}' for i in range(cuts-1, -1, -1)] )))
    return aux[['qinc_front', 'qinc_back']+[f'qinc_back_{i}' for i in range(cuts)]]

## Engine

In [ ]:
#export
def run_bifacialvf_simulation(data: DataFrame,  
                          pvarray_parameters:dict={'rtr':8.,  'cellRows': 7, 'albedo':0.4}, 
                          gps_data:dict=CHAMBERY):
    "Run bifacialvf on data, with pvarray_parameters at location"
    outfile = 'output.csv'
    pvarray_parameters.update({'sam_header':False})
    bifacialvf.simulate_inner(rename_cols(data), gps_data, outfile, **pvarray_parameters)
    return (pd.read_csv(outfile, header=2, index_col='date', parse_dates=True)
            .pipe(format_output_as_pvfactor, cuts=pvarray_parameters['cellRows']))

In [ ]:
output = run_bifacialvf_simulation(data['June 2019'], gps_data=CHAMBERY, )

  0%|          | 0/720 [00:00<?, ?it/s]/home/tc256760/Apps/bifacialvf/bifacialvf/vf.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  projectedX2 = PcellX + np.float64(PcellY) / math.tan(startElvDown);      # Projection of ElvDown to ground in +x direction (X1 and X2 opposite nomenclature for front irradiance method)
  2%|▏         | 17/720 [00:00<00:04, 161.21it/s]

 
********* 
Running Simulation for TMY3:  Chambery
Location:   Chambery
Lat:  45.637001  Long:  5.881  Tz  -1.0
Parameters: beta:  0   Sazm:  180   Height:  0.5   rtr separation:  8.0   Row type:  interior   Albedo:  0.4
Saving into output.csv
 
 


100%|██████████| 720/720 [00:03<00:00, 224.07it/s]


Finished


In [ ]:
output.qinc_back_mean.plot(figsize=(12,5))

AttributeError: 'DataFrame' object has no attribute 'qinc_back_mean'

# Exports -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()